In [ ]:
# Data Cleaning
import pandas as pd

df = pd.read_stata('SIPP_Paid_Leave.dta')

rename_dict = {
    'ssuid': 'Sample_Unit_Identifier',
    'spanel': 'Sample_Code_Panel_Year',
    'swave': 'Wave_of_Data_Collection',
    'srefmon': 'Reference_Month',
    'rhcalmn': 'Calendar_Month',
    'rhcalyr': 'Calendar_Year',
    'tfipsst': 'FIPS_State_Code',
    'epppnum': 'Person_Number',
    'esex': 'Sex',
    'wpfinwgt': 'Person_Weight',
    'tage': 'Age',
    'eeducate': 'Education_Level',
    'rmesr': 'Labour_Market_Participation',
    'birth_month': 'Birth_Month_Year'
}
df.rename(columns=rename_dict, inplace=True)

df.info()

print("\nSummary statistics for numerical columns:\n")
print(df.describe())

In [ ]:
# The initial simple difference-in-difference code is obtained from the original paper Byker (2016)

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

df = pd.read_stata('SIPP_Paid_Leave.dta')

# Convert data types to match the original dataset
# 'ssuid' should be a string of length 12
df['ssuid'] = df['ssuid'].astype(str)

# Ensure integer types for relevant columns
df['spanel'] = df['spanel'].astype(int)
df['epppnum'] = df['epppnum'].astype(int)
df['tfipsst'] = df['tfipsst'].astype('int8')
df['rhcalyr'] = df['rhcalyr'].astype(int)
df['swave'] = df['swave'].astype('int8')
df['srefmon'] = df['srefmon'].astype('int8')
df['rhcalmn'] = df['rhcalmn'].astype('int8')
df['tage'] = df['tage'].astype('int8')
df['eeducate'] = df['eeducate'].astype('int8')
df['rmesr'] = df['rmesr'].astype('int8')
df['esex'] = df['esex'].astype('int8')

# Double type column
df['wpfinwgt'] = df['wpfinwgt'].astype(float)

# Sort the data
df.sort_values(by=['ssuid', 'epppnum', 'spanel', 'swave', 'srefmon'], inplace=True)

# Create a unique ID for each individual
df['sippid'] = df.groupby(['spanel', 'ssuid', 'epppnum']).ngroup() + 1

# Generate 'months' variable (as float)
df['months'] = df.groupby('sippid').cumcount() + 1

# Convert 'birth_month' to a Period type for monthly representation
df['birth_month'] = pd.to_datetime(df['birth_month'], format='%Y-%m').dt.to_period('M')

# Create 'date' variable using year and month and convert to %tm format
df['date'] = pd.to_datetime(dict(year=df['rhcalyr'], month=df['rhcalmn'], day=1)).dt.to_period('M')

# Generate the 'birth' variable that indicates month relative to birth
df['birth'] = (df['date'] - df['birth_month']).apply(lambda x: x.n if not pd.isna(x) else np.nan)
df['birth'] = df['birth'].astype(float)

# Handle missing birth observations
df['birth_seen_f'] = (df['birth'] == 0).astype(int)
df['birth_seen'] = df.groupby('sippid')['birth_seen_f'].transform('max').astype(int)

# Find the earliest 'birth' value for each individual where birth > 0 and birth not seen
def min_birth(x):
    cond = (x['birth'] > 0) & (x['birth_seen'] == 0)
    return x.loc[cond, 'birth'].min() if cond.any() else np.nan

df['ref_month_ns'] = df.groupby('sippid').apply(min_birth).reset_index(level=0, drop=True).astype(float)
df['ref_month'] = np.nan
df.loc[(df['birth'] == 0) & (df['birth_seen'] == 1), 'ref_month'] = 1
df.loc[(df['ref_month_ns'] == df['birth']) & (df['birth_seen'] == 0), 'ref_month'] = 1
df['ref_month'] = df['ref_month'].astype(float)

# Map state codes to state names and make it categorical
state_labels = {6: "California", 34: "New Jersey", 12: "Florida", 48: "Texas", 36: "New York"}
df['state'] = df['tfipsst'].map(state_labels)
df['state'] = df['state'].astype('category')

# Find the last date for each individual and format '%tm'
df['end_date'] = df.groupby('sippid')['date'].transform('max')

# Get the weight from the last observation
df['end_weight_f'] = df.apply(lambda x: x['wpfinwgt'] if x['date'] == x['end_date'] else np.nan, axis=1).astype(float)
df['end_weight'] = df.groupby('sippid')['end_weight_f'].transform('max').astype(float)

# Round the weights
df['end_weight'] = df['end_weight'].round()

# Define policy implementation dates as period '%tm'
df['CA_date'] = pd.Period('2004-07', freq='M')
df['NJ_date'] = pd.Period('2009-07', freq='M')

# Generate variable indicating births that occurred when a paid leave was in effect in the mother's state
df['post_policy'] = 0
df.loc[(df['tfipsst'] == 34) & (df['birth_month'] >= df['NJ_date']), 'post_policy'] = 1
df.loc[(df['tfipsst'] == 6) & (df['birth_month'] >= df['CA_date']), 'post_policy'] = 1

# Labor force participation variable (treated as categorical for difference-in-difference model)
df['rm_lfp'] = (df['rmesr'] <= 7).astype(int)
df.loc[df['rmesr'] == -1, 'rm_lfp'] = np.nan

# Create difference-in-difference variables
df['treated'] = ((df['state'].isin(['California', 'New Jersey'])) & (df['post_policy'] == 1)).astype(int)

# Drop rows with missing values to avoid issues in regression
df = df.dropna(subset=['rm_lfp', 'treated', 'state', 'post_policy'])

# Difference-in-Difference regression model
formula = 'rm_lfp ~ treated + C(state) + post_policy + treated:post_policy'
model = smf.ols(formula, data=df).fit(cov_type='cluster', cov_kwds={'groups': df['sippid']})

# Print the summary of the regression model
print(model.summary())


In [ ]:
# Logit Regression

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

df = pd.read_stata('SIPP_Paid_Leave.dta')


df['ssuid'] = df['ssuid'].astype(str)
df['spanel'] = df['spanel'].astype(int)
df['epppnum'] = df['epppnum'].astype(int)
df['tfipsst'] = df['tfipsst'].astype('int8')
df['rhcalyr'] = df['rhcalyr'].astype(int)
df['swave'] = df['swave'].astype('int8')
df['srefmon'] = df['srefmon'].astype('int8')
df['rhcalmn'] = df['rhcalmn'].astype('int8')
df['tage'] = df['tage'].astype('int8')
df['eeducate'] = df['eeducate'].astype('int8')
df['rmesr'] = df['rmesr'].astype('int8')
df['esex'] = df['esex'].astype('int8')
df['wpfinwgt'] = df['wpfinwgt'].astype(float)
l
df.sort_values(by=['ssuid', 'epppnum', 'spanel', 'swave', 'srefmon'], inplace=True)
df['sippid'] = df.groupby(['spanel', 'ssuid', 'epppnum']).ngroup() + 1


df['months'] = df.groupby('sippid').cumcount() + 1


df['birth_month'] = pd.to_datetime(df['birth_month'], format='%Y-%m').dt.to_period('M')
df['date'] = pd.to_datetime(dict(year=df['rhcalyr'], month=df['rhcalmn'], day=1)).dt.to_period('M')


df['birth'] = (df['date'] - df['birth_month']).apply(lambda x: x.n if not pd.isna(x) else np.nan)
df['birth'] = df['birth'].astype(float)


df['birth_seen_f'] = (df['birth'] == 0).astype(int)
df['birth_seen'] = df.groupby('sippid')['birth_seen_f'].transform('max').astype(int)


def min_birth(x):
    cond = (x['birth'] > 0) & (x['birth_seen'] == 0)
    return x.loc[cond, 'birth'].min() if cond.any() else np.nan

df['ref_month_ns'] = df.groupby('sippid').apply(min_birth).reset_index(level=0, drop=True).astype(float)
df['ref_month'] = np.nan
df.loc[(df['birth'] == 0) & (df['birth_seen'] == 1), 'ref_month'] = 1
df.loc[(df['ref_month_ns'] == df['birth']) & (df['birth_seen'] == 0), 'ref_month'] = 1
df['ref_month'] = df['ref_month'].astype(float)

state_labels = {6: "California", 34: "New Jersey", 12: "Florida", 48: "Texas", 36: "New York"}
df['state'] = df['tfipsst'].map(state_labels)
df['state'] = df['state'].astype('category')


df['end_date'] = df.groupby('sippid')['date'].transform('max')
df['end_weight_f'] = df.apply(lambda x: x['wpfinwgt'] if x['date'] == x['end_date'] else np.nan, axis=1).astype(float)
df['end_weight'] = df.groupby('sippid')['end_weight_f'].transform('max').astype(float)
df['end_weight'] = df['end_weight'].round()

df['CA_date'] = pd.Period('2004-07', freq='M')
df['NJ_date'] = pd.Period('2009-07', freq='M')
df['post_policy'] = 0
df.loc[(df['tfipsst'] == 34) & (df['birth_month'] >= df['NJ_date']), 'post_policy'] = 1
df.loc[(df['tfipsst'] == 6) & (df['birth_month'] >= df['CA_date']), 'post_policy'] = 1


df['rm_lfp'] = (df['rmesr'] <= 7).astype(int)
df.loc[df['rmesr'] == -1, 'rm_lfp'] = np.nan

# Create difference-in-difference variables
df['treated'] = ((df['state'].isin(['California', 'New Jersey'])) & (df['post_policy'] == 1)).astype(int)

# Drop rows with missing values to avoid issues in regression
df = df.dropna(subset=['rm_lfp', 'treated', 'state', 'post_policy'])

# Run Logit regression model for Difference-in-Difference without clustered standard errors
formula = 'rm_lfp ~ treated:post_policy + C(state)'
try:
    model = smf.logit(formula, data=df).fit()
    # Print the summary of the regression model
    print(model.summary())
except np.linalg.LinAlgError:
    # If the Hessian is singular, run without clustering
    model = smf.logit(formula, data=df).fit(method='bfgs')
    print(model.summary())


In [ ]:
# Logit Regression - Subsample Analyses

!pip install stargazer
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from stargazer.stargazer import Stargazer


df = pd.read_stata('SIPP_Paid_Leave.dta')


df['ssuid'] = df['ssuid'].astype(str)
df['spanel'] = df['spanel'].astype(int)
df['epppnum'] = df['epppnum'].astype(int)
df['tfipsst'] = df['tfipsst'].astype('int8')
df['rhcalyr'] = df['rhcalyr'].astype(int)
df['swave'] = df['swave'].astype('int8')
df['srefmon'] = df['srefmon'].astype('int8')
df['rhcalmn'] = df['rhcalmn'].astype('int8')
df['tage'] = df['tage'].astype('int8')
df['eeducate'] = df['eeducate'].astype('int8')
df['rmesr'] = df['rmesr'].astype('int8')
df['esex'] = df['esex'].astype('int8')
df['wpfinwgt'] = df['wpfinwgt'].astype(float)


df.sort_values(by=['ssuid', 'epppnum', 'spanel', 'swave', 'srefmon'], inplace=True)
df['sippid'] = df.groupby(['spanel', 'ssuid', 'epppnum']).ngroup() + 1


df['months'] = df.groupby('sippid').cumcount() + 1


df['birth_month'] = pd.to_datetime(df['birth_month'], format='%Y-%m').dt.to_period('M')
df['date'] = pd.to_datetime(dict(year=df['rhcalyr'], month=df['rhcalmn'], day=1)).dt.to_period('M')


df['birth'] = (df['date'] - df['birth_month']).apply(lambda x: x.n if not pd.isna(x) else np.nan)
df['birth'] = df['birth'].astype(float)


df['birth_seen_f'] = (df['birth'] == 0).astype(int)
df['birth_seen'] = df.groupby('sippid')['birth_seen_f'].transform('max').astype(int)


def min_birth(x):
    cond = (x['birth'] > 0) & (x['birth_seen'] == 0)
    return x.loc[cond, 'birth'].min() if cond.any() else np.nan

df['ref_month_ns'] = df.groupby('sippid').apply(min_birth).reset_index(level=0, drop=True).astype(float)
df['ref_month'] = np.nan
df.loc[(df['birth'] == 0) & (df['birth_seen'] == 1), 'ref_month'] = 1
df.loc[(df['ref_month_ns'] == df['birth']) & (df['birth_seen'] == 0), 'ref_month'] = 1
df['ref_month'] = df['ref_month'].astype(float)


state_labels = {6: "California", 34: "New Jersey", 12: "Florida", 48: "Texas", 36: "New York"}
df['state'] = df['tfipsst'].map(state_labels)
df['state'] = df['state'].astype('category')

df['end_date'] = df.groupby('sippid')['date'].transform('max')
df['end_weight_f'] = df.apply(lambda x: x['wpfinwgt'] if x['date'] == x['end_date'] else np.nan, axis=1).astype(float)
df['end_weight'] = df.groupby('sippid')['end_weight_f'].transform('max').astype(float)
df['end_weight'] = df['end_weight'].round()


df['CA_date'] = pd.Period('2004-07', freq='M')
df['NJ_date'] = pd.Period('2009-07', freq='M')
df['post_policy'] = 0
df.loc[(df['tfipsst'] == 34) & (df['birth_month'] >= df['NJ_date']), 'post_policy'] = 1
df.loc[(df['tfipsst'] == 6) & (df['birth_month'] >= df['CA_date']), 'post_policy'] = 1

df['rm_lfp'] = (df['rmesr'] <= 7).astype(int)
df.loc[df['rmesr'] == -1, 'rm_lfp'] = np.nan


df['treated'] = ((df['state'].isin(['California', 'New Jersey'])) & (df['post_policy'] == 1)).astype(int)


df = df.dropna(subset=['rm_lfp', 'treated', 'state', 'post_policy'])

# Run Logit regression model for Difference-in-Difference without clustered standard errors
formula = 'rm_lfp ~ treated:post_policy + C(state)'
try:
    model = smf.logit(formula, data=df).fit()
    # Print the summary of the regression model
    print(model.summary())
except np.linalg.LinAlgError:
    # If the Hessian is singular, run without clustering
    model = smf.logit(formula, data=df).fit(method='bfgs')
    print(model.summary())


stargazer = Stargazer([model])
stargazer.title('Logit Regression Results for Labor Force Participation')


latex_table = stargazer.render_latex()
print(latex_table)

In [ ]:
# Logit Regression Analysis for Subsets of Data

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from stargazer.stargazer import Stargazer


df = pd.read_stata('SIPP_Paid_Leave.dta')


df['ssuid'] = df['ssuid'].astype(str)
df['spanel'] = df['spanel'].astype(int)
df['epppnum'] = df['epppnum'].astype(int)
df['tfipsst'] = df['tfipsst'].astype('int8')
df['rhcalyr'] = df['rhcalyr'].astype(int)
df['swave'] = df['swave'].astype('int8')
df['srefmon'] = df['srefmon'].astype('int8')
df['rhcalmn'] = df['rhcalmn'].astype('int8')
df['tage'] = df['tage'].astype('int8')
df['eeducate'] = df['eeducate'].astype('int8')
df['rmesr'] = df['rmesr'].astype('int8')
df['esex'] = df['esex'].astype('int8')
df['wpfinwgt'] = df['wpfinwgt'].astype(float)


df.sort_values(by=['ssuid', 'epppnum', 'spanel', 'swave', 'srefmon'], inplace=True)
df['sippid'] = df.groupby(['spanel', 'ssuid', 'epppnum']).ngroup() + 1


df['months'] = df.groupby('sippid').cumcount() + 1


df['birth_month'] = pd.to_datetime(df['birth_month'], format='%Y-%m').dt.to_period('M')
df['date'] = pd.to_datetime(dict(year=df['rhcalyr'], month=df['rhcalmn'], day=1)).dt.to_period('M')


df['birth'] = (df['date'] - df['birth_month']).apply(lambda x: x.n if not pd.isna(x) else np.nan)
df['birth'] = df['birth'].astype(float)


df['birth_seen_f'] = (df['birth'] == 0).astype(int)
df['birth_seen'] = df.groupby('sippid')['birth_seen_f'].transform('max').astype(int)


def min_birth(x):
    cond = (x['birth'] > 0) & (x['birth_seen'] == 0)
    return x.loc[cond, 'birth'].min() if cond.any() else np.nan

df['ref_month_ns'] = df.groupby('sippid').apply(min_birth).reset_index(level=0, drop=True).astype(float)
df['ref_month'] = np.nan
df.loc[(df['birth'] == 0) & (df['birth_seen'] == 1), 'ref_month'] = 1
df.loc[(df['ref_month_ns'] == df['birth']) & (df['birth_seen'] == 0), 'ref_month'] = 1
df['ref_month'] = df['ref_month'].astype(float)


state_labels = {6: "California", 34: "New Jersey", 12: "Florida", 48: "Texas", 36: "New York"}
df['state'] = df['tfipsst'].map(state_labels)
df['state'] = df['state'].astype('category')


df['end_date'] = df.groupby('sippid')['date'].transform('max')
df['end_weight_f'] = df.apply(lambda x: x['wpfinwgt'] if x['date'] == x['end_date'] else np.nan, axis=1).astype(float)
df['end_weight'] = df.groupby('sippid')['end_weight_f'].transform('max').astype(float)
df['end_weight'] = df['end_weight'].round()


df['CA_date'] = pd.Period('2004-07', freq='M')
df['NJ_date'] = pd.Period('2009-07', freq='M')
df['post_policy'] = 0
df.loc[(df['tfipsst'] == 34) & (df['birth_month'] >= df['NJ_date']), 'post_policy'] = 1
df.loc[(df['tfipsst'] == 6) & (df['birth_month'] >= df['CA_date']), 'post_policy'] = 1


df['rm_lfp'] = (df['rmesr'] <= 7).astype(int)
df.loc[df['rmesr'] == -1, 'rm_lfp'] = np.nan


df['treated'] = ((df['state'].isin(['California', 'New Jersey'])) & (df['post_policy'] == 1)).astype(int)


df = df.dropna(subset=['rm_lfp', 'treated', 'state', 'post_policy'])


formula = 'rm_lfp ~ treated:post_policy + C(state)'
try:
    model = smf.logit(formula, data=df).fit()
    # Print the summary of the regression model
    print(model.summary())
except np.linalg.LinAlgError:
    # If the Hessian is singular, run without clustering
    model = smf.logit(formula, data=df).fit(method='bfgs')
    print(model.summary())


stargazer = Stargazer([model])
stargazer.title('Logit Regression Results for Labor Force Participation')


latex_table = stargazer.render_latex()
print(latex_table)

# Group education levels
def categorize_education(edu_code):
    if edu_code in range(32, 39):
        return 'Less than High School'
    elif edu_code == 39:
        return 'High School Graduate'
    elif edu_code in range(40, 42):
        return 'Some College'
    elif edu_code in range(42, 44):
        return 'Associate Degree'
    elif edu_code == 44:
        return 'Bachelor Degree'
    elif edu_code in range(45, 48):
        return 'Graduate Degree'
    else:
        return np.nan

df['education_category'] = df['eeducate'].apply(categorize_education)


df = df.dropna(subset=['education_category', 'rm_lfp'])

# Additional Subsample Regression Models
education_subsets = {
    'Less than High School': df[df['education_category'] == 'Less than High School'],
    'High School Graduate': df[df['education_category'] == 'High School Graduate'],
    'Some College': df[df['education_category'] == 'Some College'],
    'Associate Degree': df[df['education_category'] == 'Associate Degree'],
    'Bachelor Degree': df[df['education_category'] == 'Bachelor Degree'],
    'Graduate Degree': df[df['education_category'] == 'Graduate Degree']
}

age_subsets = {
    'Age 24-35': df[(df['tage'] >= 24) & (df['tage'] <= 35)],
    'Age 36-45': df[(df['tage'] > 35) & (df['tage'] <= 45)]
}

education_models = []
for subset_name, subset_data in education_subsets.items():
    try:
        subset_model = smf.logit(formula, data=subset_data).fit()
        education_models.append(subset_model)
        print(f"\nRegression results for {subset_name} subset:\n")
        print(subset_model.summary())
    except np.linalg.LinAlgError:
        subset_model = smf.logit(formula, data=subset_data).fit(method='bfgs')
        education_models.append(subset_model)
        print(f"\nRegression results for {subset_name} subset (using BFGS):\n")
        print(subset_model.summary())


stargazer_education = Stargazer(education_models)
stargazer_education.title('Logit Regression Results for Subsamples Classified by Education Level')
stargazer_education.custom_columns(['Less than High School', 'High School Graduate', 'Some College', 'Associate Degree', 'Bachelor Degree', 'Graduate Degree'], [1, 1, 1, 1, 1, 1])
latex_table_education = stargazer_education.render_latex()
print(latex_table_education)

age_models = []
for subset_name, subset_data in age_subsets.items():
    try:
        subset_model = smf.logit(formula, data=subset_data).fit()
        age_models.append(subset_model)
        print(f"\nRegression results for {subset_name} subset:\n")
        print(subset_model.summary())
    except np.linalg.LinAlgError:
        subset_model = smf.logit(formula, data=subset_data).fit(method='bfgs')
        age_models.append(subset_model)
        print(f"\nRegression results for {subset_name} subset (using BFGS):\n")
        print(subset_model.summary())


stargazer_age = Stargazer(age_models)
stargazer_age.title('Logit Regression Results for Subsamples Classified by Age')
stargazer_age.custom_columns(['Age 24-35', 'Age 36-45'], [1, 1])
latex_table_age = stargazer_age.render_latex()
print(latex_table_age)


In [ ]:
# Summary Statistics

import matplotlib.pyplot as plt
import seaborn as sns

# Summary Statistics for Labor Force Participation (rm_lfp)
print("Summary Statistics for Labor Force Participation:")
print("\nFrequency counts:")
print(df['rm_lfp'].value_counts(dropna=False))

# Visualization of Labor Force Participation
plt.figure(figsize=(10, 6))
sns.countplot(x='rm_lfp', data=df, palette='Blues')
plt.title('Comparative Analysis of Female Labor Force Non-Participation and Participation in the Entire Population')
plt.xlabel('Labor Force Participation (Attachment)')
plt.ylabel('Number of Women (Entire Population)')
plt.xticks([0, 1], ['Not in Labour Force (Unattached)', 'In Labour Force (Attached)'])


for p in plt.gca().patches:
    plt.text(p.get_x() + p.get_width() / 2., p.get_height(), f'{int(p.get_height())}', ha='center', va='bottom')

plt.show()


def categorize_education(edu_code):
    if edu_code in range(32, 39):
        return 'Less than High School'
    elif edu_code == 39:
        return 'High School Graduate'
    elif edu_code in range(40, 42):
        return 'Some College'
    elif edu_code in range(42, 44):
        return 'Associate Degree'
    elif edu_code == 44:
        return 'Bachelor\'s Degree'
    elif edu_code in range(45, 48):
        return 'Graduate Degree'
    else:
        return np.nan

df['education_category'] = df['eeducate'].apply(categorize_education)


df = df.dropna(subset=['education_category', 'rm_lfp'])


education_lfp = df.groupby('education_category')['rm_lfp'].mean().reset_index()


education_order = [
    'Less than High School',
    'High School Graduate',
    'Some College',
    'Associate Degree',
    'Bachelor\'s Degree',
    'Graduate Degree'
]


plt.figure(figsize=(12, 6))
sns.barplot(x='education_category', y='rm_lfp', data=education_lfp, order=education_order, palette='Reds')
plt.title('Labor Force Participation Rate by Education Level')
plt.xlabel('Education Level of Women')
plt.ylabel('Labor Force Participation Rate')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Summary Stats: Line Graph - Trends Over Time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


df = pd.read_stata('SIPP_Paid_Leave.dta')


df['ssuid'] = df['ssuid'].astype(str)
df['spanel'] = df['spanel'].astype(int)
df['epppnum'] = df['epppnum'].astype(int)
df['tfipsst'] = df['tfipsst'].astype('int8')
df['rhcalyr'] = df['rhcalyr'].astype(int)
df['swave'] = df['swave'].astype('int8')
df['srefmon'] = df['srefmon'].astype('int8')
df['rhcalmn'] = df['rhcalmn'].astype('int8')
df['tage'] = df['tage'].astype('int8')
df['eeducate'] = df['eeducate'].astype('int8')
df['rmesr'] = df['rmesr'].astype('int8')
df['esex'] = df['esex'].astype('int8')
df['wpfinwgt'] = df['wpfinwgt'].astype(float)


df.sort_values(by=['ssuid', 'epppnum', 'spanel', 'swave', 'srefmon'], inplace=True)
df['sippid'] = df.groupby(['spanel', 'ssuid', 'epppnum']).ngroup() + 1


df['birth_month'] = pd.to_datetime(df['birth_month'], format='%Y-%m').dt.to_period('M')
df['date'] = pd.to_datetime(dict(year=df['rhcalyr'], month=df['rhcalmn'], day=1))


df['CA_date'] = pd.Period('2004-07', freq='M')
df['NJ_date'] = pd.Period('2009-07', freq='M')
df['post_policy'] = 0


df.loc[(df['tfipsst'] == 34) & (df['birth_month'] >= df['NJ_date']), 'post_policy'] = 1
df.loc[(df['tfipsst'] == 6) & (df['birth_month'] >= df['CA_date']), 'post_policy'] = 1


df['rm_lfp'] = (df['rmesr'] <= 7).astype(int)
df.loc[df['rmesr'] == -1, 'rm_lfp'] = np.nan


state_labels = {6: "California", 34: "New Jersey", 12: "Florida", 48: "Texas", 36: "New York"}
df['state'] = df['tfipsst'].map(state_labels)
df['state'] = df['state'].astype('category')


df = df.dropna(subset=['rm_lfp', 'state', 'post_policy'])

# Filter only for relevant states (California, New Jersey, Florida, Texas, New York)
relevant_states = ["California", "New Jersey", "Florida", "Texas", "New York"]
df = df[df['state'].isin(relevant_states)]

# Calculate average labor force participation rate by date and state
df_time_summary = df.groupby(['date', 'state'])['rm_lfp'].mean().reset_index()

# Plot for California as treated and others as controls (excluding New Jersey)
plt.figure(figsize=(14, 8))
for state in relevant_states:
    if state == "California":
        plt.plot(df_time_summary[df_time_summary['state'] == state]['date'],
                 df_time_summary[df_time_summary['state'] == state]['rm_lfp'],
                 label=state, color='red')
    elif state != "New Jersey":
        plt.plot(df_time_summary[df_time_summary['state'] == state]['date'],
                 df_time_summary[df_time_summary['state'] == state]['rm_lfp'],
                 label=state, color='orange', linestyle='--')
plt.axvline(pd.Timestamp('2004-07-01'), color='green', linestyle='--', label='Policy Implementation (July 2004)')
plt.title('Labor Force Participation: Trends in California v/s Control States')
plt.xlabel('Date')
plt.ylabel('Labor Force Participation Rate')
plt.legend()
plt.show()

# Plot for New Jersey as treated and others as controls (excluding California)
plt.figure(figsize=(14, 8))
for state in relevant_states:
    if state == "New Jersey":
        plt.plot(df_time_summary[df_time_summary['state'] == state]['date'],
                 df_time_summary[df_time_summary['state'] == state]['rm_lfp'],
                 label=state, color='blue')
    elif state != "California":
        plt.plot(df_time_summary[df_time_summary['state'] == state]['date'],
                 df_time_summary[df_time_summary['state'] == state]['rm_lfp'],
                 label=state, color='orange', linestyle='--')
plt.axvline(pd.Timestamp('2009-07-01'), color='green', linestyle='--', label='Policy Implementation (July 2009)')
plt.title('Labor Force Participation: Trends in New Jersey v/s Control States')
plt.xlabel('Date')
plt.ylabel('Labor Force Participation Rate')
plt.legend()
plt.show()


In [ ]:
# Lasso Model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


alphas = np.logspace(-6, 6, 100)
coefs = []


for alpha in alphas:
    lasso_model = LogisticRegression(
        penalty='l1',
        C=1/alpha,  # C is the inverse of alpha in sklearn
        solver='saga',
        random_state=42,
        max_iter=500
    )
    lasso_model.fit(X_scaled, y)
    coefs.append(lasso_model.coef_[0])

coefs = np.array(coefs)


label_mapping = {
    'treated_post_policy': 'Treated*PostPolicy',
    'tage': 'Age',
    'eeducate': 'Level of Education',
    'esex': 'Sex',
    'state_Florida': 'Florida',
    'state_New Jersey': 'New Jersey',
    'state_New York': 'New York',
    'state_Texas': 'Texas',
    'state_California': 'California'
}


fig, ax = plt.subplots(figsize=(8, 8))
for i in range(coefs.shape[1]):
    label = label_mapping.get(X.columns[i], X.columns[i])  # Use the mapping or the original name if not found
    ax.plot(-np.log10(alphas), coefs[:, i], label=label)

ax.legend(loc='upper left')
ax.set_xlabel('$-\\log(\\lambda)$', fontsize=20)
ax.set_ylabel('Standardized coefficients', fontsize=20)
plt.title('Lasso Logistic Coefficient Paths', fontsize=20)
plt.show()


mean_scores = []
std_scores = []

for alpha in alphas:
    lasso_model = LogisticRegression(
        penalty='l1',
        C=1/alpha,
        solver='saga',
        random_state=42,
        max_iter=500
    )
    scores = cross_val_score(lasso_model, X_scaled, y, cv=5, scoring='accuracy')
    mean_scores.append(scores.mean())
    std_scores.append(scores.std())

fig, ax = plt.subplots(figsize=(8, 8))
ax.errorbar(
    -np.log10(alphas),
    mean_scores,
    yerr=std_scores,
    fmt='o',
    ecolor='lightgray',
    elinewidth=2,
    capsize=4,
    color='blue'
)
best_alpha_idx = np.argmax(mean_scores)
ax.axvline(-np.log10(alphas[best_alpha_idx]), color='red', linestyle='--', label='Best $\\lambda$')
ax.set_xlabel('$-\\log(\\lambda)$', fontsize=20)
ax.set_ylabel('Cross-validated Accuracy', fontsize=20)
plt.title('Lasso Logistic Regression Cross-Validated Accuracy', fontsize=20)
ax.legend()
plt.show()


In [ ]:
# Ridge Model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


alphas = np.logspace(-6, 6, 100)
coefs = []


for alpha in alphas:
    ridge_model = LogisticRegression(
        penalty='l2',
        C=1/alpha,  # C is the inverse of alpha in sklearn
        solver='saga',
        random_state=42,
        max_iter=500
    )
    ridge_model.fit(X_scaled, y)
    coefs.append(ridge_model.coef_[0])

coefs = np.array(coefs)


label_mapping = {
    'treated_post_policy': 'Treated*PostPolicy',
    'tage': 'Age',
    'eeducate': 'Level of Education',
    'esex': 'Sex',
    'state_Florida': 'Florida',
    'state_New Jersey': 'New Jersey',
    'state_New York': 'New York',
    'state_Texas': 'Texas',
    'state_California': 'California'
}


fig, ax = plt.subplots(figsize=(8, 8))
for i in range(coefs.shape[1]):
    label = label_mapping.get(X.columns[i], X.columns[i])  # Use the mapping or the original name if not found
    ax.plot(-np.log10(alphas), coefs[:, i], label=label)

ax.legend(loc='upper left')
ax.set_xlabel('$-\\log(\\lambda)$', fontsize=20)
ax.set_ylabel('Standardized coefficients', fontsize=20)
plt.title('Ridge Logistic Coefficient Paths', fontsize=20)
plt.show()


from sklearn.model_selection import cross_val_score

mean_scores = []
std_scores = []

for alpha in alphas:
    ridge_model = LogisticRegression(
        penalty='l2',
        C=1/alpha,
        solver='saga',
        random_state=42,
        max_iter=500
    )
    scores = cross_val_score(ridge_model, X_scaled, y, cv=5, scoring='accuracy')
    mean_scores.append(scores.mean())
    std_scores.append(scores.std())

fig, ax = plt.subplots(figsize=(8, 8))
ax.errorbar(
    -np.log10(alphas),
    mean_scores,
    yerr=std_scores,
    fmt='o',
    ecolor='lightgray',
    elinewidth=2,
    capsize=4,
    color='blue'
)
best_alpha_idx = np.argmax(mean_scores)
ax.axvline(-np.log10(alphas[best_alpha_idx]), color='red', linestyle='--', label='Best $\\lambda$')
ax.set_xlabel('$-\\log(\\lambda)$', fontsize=20)
ax.set_ylabel('Cross-validated Accuracy', fontsize=20)
plt.title('Ridge Logistic Regression Cross-Validated Accuracy', fontsize=20)
ax.legend()
plt.show()



In [ ]:
# Random Forest Model

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error


df = pd.read_stata('SIPP_Paid_Leave.dta')


df['ssuid'] = df['ssuid'].astype(str)
df['spanel'] = df['spanel'].astype(int)
df['epppnum'] = df['epppnum'].astype(int)
df['tfipsst'] = df['tfipsst'].astype('int8')
df['rhcalyr'] = df['rhcalyr'].astype(int)
df['swave'] = df['swave'].astype('int8')
df['srefmon'] = df['srefmon'].astype('int8')
df['rhcalmn'] = df['rhcalmn'].astype('int8')
df['tage'] = df['tage'].astype('int8')
df['eeducate'] = df['eeducate'].astype('int8')
df['rmesr'] = df['rmesr'].astype('int8')
df['esex'] = df['esex'].astype('int8')
df['wpfinwgt'] = df['wpfinwgt'].astype(float)


df.sort_values(by=['ssuid', 'epppnum', 'spanel', 'swave', 'srefmon'], inplace=True)
df['sippid'] = df.groupby(['spanel', 'ssuid', 'epppnum']).ngroup() + 1


df['months'] = df.groupby('sippid').cumcount() + 1


df['birth_month'] = pd.to_datetime(df['birth_month'], format='%Y-%m').dt.to_period('M')
df['date'] = pd.to_datetime(dict(year=df['rhcalyr'], month=df['rhcalmn'], day=1)).dt.to_period('M')


df['birth'] = (df['date'] - df['birth_month']).apply(lambda x: x.n if not pd.isna(x) else np.nan)
df['birth'] = df['birth'].astype(float)


df['birth_seen_f'] = (df['birth'] == 0).astype(int)
df['birth_seen'] = df.groupby('sippid')['birth_seen_f'].transform('max').astype(int)


def min_birth(x):
    cond = (x['birth'] > 0) & (x['birth_seen'] == 0)
    return x.loc[cond, 'birth'].min() if cond.any() else np.nan

df['ref_month_ns'] = df.groupby('sippid').apply(min_birth).reset_index(level=0, drop=True).astype(float)
df['ref_month'] = np.nan
df.loc[(df['birth'] == 0) & (df['birth_seen'] == 1), 'ref_month'] = 1
df.loc[(df['ref_month_ns'] == df['birth']) & (df['birth_seen'] == 0), 'ref_month'] = 1
df['ref_month'] = df['ref_month'].astype(float)


state_labels = {
    6: "California",
    34: "New Jersey",
    12: "Florida",
    48: "Texas",
    36: "New York"
}
df['state'] = df['tfipsst'].map(state_labels)
df['state'] = df['state'].astype('category')


df['CA_date'] = pd.Period('2004-07', freq='M')
df['NJ_date'] = pd.Period('2009-07', freq='M')
df['post_policy'] = 0
df.loc[(df['tfipsst'] == 34) & (df['birth_month'] >= df['NJ_date']), 'post_policy'] = 1
df.loc[(df['tfipsst'] == 6) & (df['birth_month'] >= df['CA_date']), 'post_policy'] = 1


df['rm_lfp'] = (df['rmesr'] <= 7).astype(int)
df.loc[df['rmesr'] == -1, 'rm_lfp'] = np.nan


df['treated'] = df['state'].isin(['California', 'New Jersey']).astype(int)
df['treated_post_policy'] = df['treated'] * df['post_policy']


df = df.dropna(subset=['rm_lfp', 'treated', 'state', 'post_policy', 'tage', 'eeducate', 'esex'])


df = pd.get_dummies(df, columns=['state'], drop_first=False)
X = df[['treated', 'treated_post_policy', 'tage', 'eeducate', 'esex'] + [col for col in df.columns if col.startswith('state_')]]
y = df['rm_lfp']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)


rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)


y_hat_RF = rf.predict(X_test)


rf_mse = mean_squared_error(y_test, y_hat_RF)
print(f'Random Forest Model Mean Squared Error: {rf_mse}')


importances = rf.feature_importances_
feature_names = ['treated', 'treated_post_policy', 'Age', 'Level of Education', 'Sex'] + [col.split('_', 1)[-1] for col in df.columns if col.startswith('state_')]

# Plot the feature importances with logarithmic scale
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(range(len(importances)), importances, tick_label=feature_names)
ax.set_yscale('log')  # Apply logarithmic scale to y-axis
ax.set_xticklabels(feature_names, rotation=45, ha='right')
ax.set_xlabel('Feature', fontsize=12)
ax.set_ylabel('Importance (Log Scale)', fontsize=12)
ax.set_title('Feature Importances', fontsize=15)
plt.tight_layout()
plt.show()



In [ ]:
# Decision Trees

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_validate, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix
import matplotlib.pyplot as plt


df = pd.read_stata('SIPP_Paid_Leave.dta')


df['ssuid'] = df['ssuid'].astype(str)
df['spanel'] = df['spanel'].astype(int)
df['epppnum'] = df['epppnum'].astype(int)
df['tfipsst'] = df['tfipsst'].astype('int8')
df['rhcalyr'] = df['rhcalyr'].astype(int)
df['swave'] = df['swave'].astype('int8')
df['srefmon'] = df['srefmon'].astype('int8')
df['rhcalmn'] = df['rhcalmn'].astype('int8')
df['tage'] = df['tage'].astype('int8')
df['eeducate'] = df['eeducate'].astype('int8')
df['rmesr'] = df['rmesr'].astype('int8')
df['esex'] = df['esex'].astype('int8')
df['wpfinwgt'] = df['wpfinwgt'].astype(float)


df.sort_values(by=['ssuid', 'epppnum', 'spanel', 'swave', 'srefmon'], inplace=True)
df['sippid'] = df.groupby(['spanel', 'ssuid', 'epppnum']).ngroup() + 1


df['months'] = df.groupby('sippid').cumcount() + 1


df['birth_month'] = pd.to_datetime(df['birth_month'], format='%Y-%m').dt.to_period('M')
df['date'] = pd.to_datetime(dict(year=df['rhcalyr'], month=df['rhcalmn'], day=1)).dt.to_period('M')


df['birth'] = (df['date'] - df['birth_month']).apply(lambda x: x.n if not pd.isna(x) else np.nan)
df['birth'] = df['birth'].astype(float)

s
df['birth_seen_f'] = (df['birth'] == 0).astype(int)
df['birth_seen'] = df.groupby('sippid')['birth_seen_f'].transform('max').astype(int)


def min_birth(x):
    cond = (x['birth'] > 0) & (x['birth_seen'] == 0)
    return x.loc[cond, 'birth'].min() if cond.any() else np.nan

df['ref_month_ns'] = df.groupby('sippid').apply(min_birth).reset_index(level=0, drop=True).astype(float)
df['ref_month'] = np.nan
df.loc[(df['birth'] == 0) & (df['birth_seen'] == 1), 'ref_month'] = 1
df.loc[(df['ref_month_ns'] == df['birth']) & (df['birth_seen'] == 0), 'ref_month'] = 1
df['ref_month'] = df['ref_month'].astype(float)


state_labels = {
    6: "California",
    34: "New Jersey",
    12: "Florida",
    48: "Texas",
    36: "New York"
}
df['state'] = df['tfipsst'].map(state_labels)
df['state'] = df['state'].astype('category')


df['CA_date'] = pd.Period('2004-07', freq='M')
df['NJ_date'] = pd.Period('2009-07', freq='M')
df['post_policy'] = 0
df.loc[(df['tfipsst'] == 34) & (df['birth_month'] >= df['NJ_date']), 'post_policy'] = 1
df.loc[(df['tfipsst'] == 6) & (df['birth_month'] >= df['CA_date']), 'post_policy'] = 1


df['rm_lfp'] = (df['rmesr'] <= 7).astype(int)
df.loc[df['rmesr'] == -1, 'rm_lfp'] = np.nan


df['treated'] = df['state'].isin(['California', 'New Jersey']).astype(int)
df['treated_post_policy'] = df['treated'] * df['post_policy']


df = df.dropna(subset=['rm_lfp', 'treated', 'state', 'post_policy'])


df = pd.get_dummies(df, columns=['state'], drop_first=False)
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.5, random_state=0)

clf = DecisionTreeClassifier(criterion='entropy', random_state=0)
clf.fit(X_train, y_train)
print("Accuracy Score (Test Set):", accuracy_score(y_test, clf.predict(X_test)))


resid_dev = np.sum(log_loss(y_train, clf.predict_proba(X_train)))
print("Residual Deviance:", resid_dev)


fig, ax = plt.subplots(figsize=(12, 12))
plot_tree(clf, feature_names=X.columns, filled=True, ax=ax)
plt.show()


print(export_text(clf, feature_names=list(X.columns), show_weights=True))

# Cross-validation using ShuffleSplit
validation = ShuffleSplit(n_splits=1, test_size=200, random_state=0)
results = cross_validate(clf, X_scaled, y, cv=validation)
print("Test Score (Cross-Validation):", results['test_score'])

# Pruning the tree using cost complexity pruning
ccp_path = clf.cost_complexity_pruning_path(X_train, y_train)
kfold = KFold(10, random_state=1, shuffle=True)
grid = GridSearchCV(clf, {'ccp_alpha': ccp_path.ccp_alphas}, refit=True, cv=kfold, scoring='accuracy')
grid.fit(X_train, y_train)
print("Best Score after Pruning:", grid.best_score_)

# Plotting the best tree after pruning
best_ = grid.best_estimator_
fig, ax = plt.subplots(figsize=(12, 12))
plot_tree(best_, feature_names=X.columns, filled=True, ax=ax)
plt.show()


print("Number of Leaves in Best Tree:", best_.tree_.n_leaves)


print("Accuracy Score (Pruned Tree, Test Set):", accuracy_score(y_test, best_.predict(X_test)))

# Confusion Matrix
confusion = confusion_matrix(y_test, best_.predict(X_test))
print("Confusion Matrix (Test Set):")
print(confusion)


In [ ]:
# Boosting

!pip install imbalanced-learn
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE


df = pd.read_stata('SIPP_Paid_Leave.dta')


df['ssuid'] = df['ssuid'].astype(str)
df['spanel'] = df['spanel'].astype(int)
df['epppnum'] = df['epppnum'].astype(int)
df['tfipsst'] = df['tfipsst'].astype('int8')
df['rhcalyr'] = df['rhcalyr'].astype(int)
df['swave'] = df['swave'].astype('int8')
df['srefmon'] = df['srefmon'].astype('int8')
df['rhcalmn'] = df['rhcalmn'].astype('int8')
df['tage'] = df['tage'].astype('int8')
df['eeducate'] = df['eeducate'].astype('int8')
df['rmesr'] = df['rmesr'].astype('int8')
df['esex'] = df['esex'].astype('int8')
df['wpfinwgt'] = df['wpfinwgt'].astype(float)

l
df.sort_values(by=['ssuid', 'epppnum', 'spanel', 'swave', 'srefmon'], inplace=True)
df['sippid'] = df.groupby(['spanel', 'ssuid', 'epppnum']).ngroup() + 1


df['months'] = df.groupby('sippid').cumcount() + 1


df['birth_month'] = pd.to_datetime(df['birth_month'], format='%Y-%m').dt.to_period('M')
df['date'] = pd.to_datetime(dict(year=df['rhcalyr'], month=df['rhcalmn'], day=1)).dt.to_period('M')


df['birth'] = (df['date'] - df['birth_month']).apply(lambda x: x.n if not pd.isna(x) else np.nan)
df['birth'] = df['birth'].astype(float)


df['birth_seen_f'] = (df['birth'] == 0).astype(int)
df['birth_seen'] = df.groupby('sippid')['birth_seen_f'].transform('max').astype(int)


def min_birth(x):
    cond = (x['birth'] > 0) & (x['birth_seen'] == 0)
    return x.loc[cond, 'birth'].min() if cond.any() else np.nan

df['ref_month_ns'] = df.groupby('sippid').apply(min_birth).reset_index(level=0, drop=True).astype(float)
df['ref_month'] = np.nan
df.loc[(df['birth'] == 0) & (df['birth_seen'] == 1), 'ref_month'] = 1
df.loc[(df['ref_month_ns'] == df['birth']) & (df['birth_seen'] == 0), 'ref_month'] = 1
df['ref_month'] = df['ref_month'].astype(float)


state_labels = {
    6: "California",
    34: "New Jersey",
    12: "Florida",
    48: "Texas",
    36: "New York"
}
df['state'] = df['tfipsst'].map(state_labels)
df['state'] = df['state'].astype('category')


df['CA_date'] = pd.Period('2004-07', freq='M')
df['NJ_date'] = pd.Period('2009-07', freq='M')
df['post_policy'] = 0
df.loc[(df['tfipsst'] == 34) & (df['birth_month'] >= df['NJ_date']), 'post_policy'] = 1
df.loc[(df['tfipsst'] == 6) & (df['birth_month'] >= df['CA_date']), 'post_policy'] = 1


df['rm_lfp'] = (df['rmesr'] <= 7).astype(int)
df.loc[df['rmesr'] == -1, 'rm_lfp'] = np.nan


df['treated'] = df['state'].isin(['California', 'New Jersey']).astype(int)
df['treated_post_policy'] = df['treated'] * df['post_policy']


df = df.dropna(subset=['rm_lfp', 'treated', 'state', 'post_policy'])


df = pd.get_dummies(df, columns=['state'], drop_first=False)
X = df[['treated', 'treated_post_policy', 'tage', 'eeducate', 'esex'] + [col for col in df.columns if col.startswith('state_')]]
y = df['rm_lfp']


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)


X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# Boosting Model
boosting_model = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.05,  # Lower learning rate for better control
    max_depth=3,
    min_samples_split=10,  # Helps control overfitting
    subsample=0.8,  # Adds regularization
    random_state=42
)
boosting_model.fit(X_train, y_train)


y_hat_boost = boosting_model.predict(X_test)


mse_boost = mean_squared_error(y_test, y_hat_boost)


accuracy_boost = accuracy_score(y_test, y_hat_boost)

print(f"Boosting Model Mean Squared Error: {mse_boost}")
print(f"Boosting Model Accuracy: {accuracy_boost}")


In [ ]:
# DAG Model

import pandas as pd
import numpy as np
!pip install dowhy
from dowhy import CausalModel
import networkx as nx
import matplotlib.pyplot as plt


df = pd.read_stata('SIPP_Paid_Leave.dta')


df['ssuid'] = df['ssuid'].astype(str)
df['spanel'] = df['spanel'].astype(int)
df['epppnum'] = df['epppnum'].astype(int)
df['tfipsst'] = df['tfipsst'].astype('int8')
df['rhcalyr'] = df['rhcalyr'].astype(int)
df['swave'] = df['swave'].astype('int8')
df['srefmon'] = df['srefmon'].astype('int8')
df['rhcalmn'] = df['rhcalmn'].astype('int8')
df['tage'] = df['tage'].astype('int8')
df['eeducate'] = df['eeducate'].astype('int8')
df['rmesr'] = df['rmesr'].astype('int8')
df['esex'] = df['esex'].astype('int8')
df['wpfinwgt'] = df['wpfinwgt'].astype(float)


df.sort_values(by=['ssuid', 'epppnum', 'spanel', 'swave', 'srefmon'], inplace=True)
df['sippid'] = df.groupby(['spanel', 'ssuid', 'epppnum']).ngroup() + 1


df['months'] = df.groupby('sippid').cumcount() + 1

e
df['birth_month'] = pd.to_datetime(df['birth_month'], format='%Y-%m').dt.to_period('M')
df['date'] = pd.to_datetime(dict(year=df['rhcalyr'], month=df['rhcalmn'], day=1)).dt.to_period('M')


df['birth'] = (df['date'] - df['birth_month']).apply(lambda x: x.n if not pd.isna(x) else np.nan)
df['birth'] = df['birth'].astype(float)


df['birth_seen_f'] = (df['birth'] == 0).astype(int)
df['birth_seen'] = df.groupby('sippid')['birth_seen_f'].transform('max').astype(int)


def min_birth(x):
    cond = (x['birth'] > 0) & (x['birth_seen'] == 0)
    return x.loc[cond, 'birth'].min() if cond.any() else np.nan

df['ref_month_ns'] = df.groupby('sippid').apply(min_birth).reset_index(level=0, drop=True).astype(float)
df['ref_month'] = np.nan
df.loc[(df['birth'] == 0) & (df['birth_seen'] == 1), 'ref_month'] = 1
df.loc[(df['ref_month_ns'] == df['birth']) & (df['birth_seen'] == 0), 'ref_month'] = 1
df['ref_month'] = df['ref_month'].astype(float)


state_labels = {
    6: "California",
    34: "New Jersey",
    12: "Florida",
    48: "Texas",
    36: "New York"
}
df['state'] = df['tfipsst'].map(state_labels)
df['state'] = df['state'].astype('category')


df['CA_date'] = pd.Period('2004-07', freq='M')
df['NJ_date'] = pd.Period('2009-07', freq='M')
df['post_policy'] = 0
df.loc[(df['tfipsst'] == 34) & (df['birth_month'] >= df['NJ_date']), 'post_policy'] = 1
df.loc[(df['tfipsst'] == 6) & (df['birth_month'] >= df['CA_date']), 'post_policy'] = 1


df['rm_lfp'] = (df['rmesr'] <= 7).astype(int)
df.loc[df['rmesr'] == -1, 'rm_lfp'] = np.nan


df['treated'] = df['state'].isin(['California', 'New Jersey']).astype(int)
df['treated_post_policy'] = df['treated'] * df['post_policy']


df = df.dropna(subset=['rm_lfp', 'treated', 'state', 'post_policy'])

# Define DAG structure using gml_graph for causal analysis
gml_graph = """
graph [
    directed 1
    node [
        id 0
        label "treated_post_policy"
    ]
    node [
        id 1
        label "rm_lfp"
    ]
    node [
        id 2
        label "tage"
    ]
    node [
        id 3
        label "eeducate"
    ]
    node [
        id 4
        label "esex"
    ]
    edge [
        source 0
        target 1
    ]
    edge [
        source 2
        target 1
    ]
    edge [
        source 3
        target 1
    ]
    edge [
        source 4
        target 1
    ]
    edge [
        source 2
        target 0
    ]
    edge [
        source 3
        target 0
    ]
    edge [
        source 4
        target 0
    ]
]
"""

# Create causal model using dowhy
model = CausalModel(
    data=df,
    treatment='treated_post_policy',
    outcome='rm_lfp',
    graph=gml_graph
)

# View the causal model graph
model.view_model()

# Perform identification, estimation, and refutation
identified_estimand = model.identify_effect()
estimate = model.estimate_effect(identified_estimand,
                                 method_name="backdoor.linear_regression")
print(estimate)

refute_results = model.refute_estimate(identified_estimand, estimate, method_name="placebo_treatment_refuter")
print(refute_results)


In [ ]:
# DAG Model Continued

import pandas as pd
import numpy as np
from dowhy import CausalModel
import pygraphviz as pgv
import matplotlib.pyplot as plt


df = pd.read_stata('SIPP_Paid_Leave.dta')


df['ssuid'] = df['ssuid'].astype(str)
df['spanel'] = df['spanel'].astype(int)
df['epppnum'] = df['epppnum'].astype(int)
df['tfipsst'] = df['tfipsst'].astype('int8')
df['rhcalyr'] = df['rhcalyr'].astype(int)
df['swave'] = df['swave'].astype('int8')
df['srefmon'] = df['srefmon'].astype('int8')
df['rhcalmn'] = df['rhcalmn'].astype('int8')
df['tage'] = df['tage'].astype('int8')
df['eeducate'] = df['eeducate'].astype('int8')
df['rmesr'] = df['rmesr'].astype('int8')
df['esex'] = df['esex'].astype('int8')
df['wpfinwgt'] = df['wpfinwgt'].astype(float)


df.sort_values(by=['ssuid', 'epppnum', 'spanel', 'swave', 'srefmon'], inplace=True)
df['sippid'] = df.groupby(['spanel', 'ssuid', 'epppnum']).ngroup() + 1


df['birth_month'] = pd.to_datetime(df['birth_month'], format='%Y-%m').dt.to_period('M')
df['date'] = pd.to_datetime(dict(year=df['rhcalyr'], month=df['rhcalmn'], day=1)).dt.to_period('M')


df['CA_date'] = pd.Period('2004-07', freq='M')
df['NJ_date'] = pd.Period('2009-07', freq='M')
df['post_policy'] = 0
df.loc[(df['tfipsst'] == 34) & (df['birth_month'] >= df['NJ_date']), 'post_policy'] = 1
df.loc[(df['tfipsst'] == 6) & (df['birth_month'] >= df['CA_date']), 'post_policy'] = 1


df['rm_lfp'] = (df['rmesr'] <= 7).astype(int)
df.loc[df['rmesr'] == -1, 'rm_lfp'] = np.nan


df['treated'] = df['tfipsst'].isin([6, 34]).astype(int)  # California (6) and New Jersey (34) are treated states
df['treated_post_policy'] = df['treated'] * df['post_policy']


df = df.dropna(subset=['rm_lfp', 'treated', 'post_policy', 'tage', 'eeducate', 'esex'])


controls = ['esex', 'tage', 'eeducate']


gml_graph = """
graph[directed 1
      node[ id "treated_post_policy" label "treated_PostPolicy" ]
      node[ id "rm_lfp" label "Labour Force Participation" ]
      node[ id "esex" label "Sex" ]
      node[ id "tage" label "Age" ]
      node[ id "eeducate" label "Education" ]
      
      edge[ source "esex" target "rm_lfp" ]
      edge[ source "tage" target "rm_lfp" ]
      edge[ source "eeducate" target "rm_lfp" ]
      edge[ source "esex" target "treated_post_policy" ]
      edge[ source "tage" target "treated_post_policy" ]
      edge[ source "eeducate" target "treated_post_policy" ]
      edge[ source "treated_post_policy" target "rm_lfp" ]
]
"""

# Create causal model using DoWhy
model = CausalModel(
    data=df,
    treatment='treated_post_policy',
    outcome='rm_lfp',
    graph=gml_graph
)

# View the causal graph
model.view_model(layout="dot")
plt.show()

# Identify the causal effect using the backdoor criterion
identified_estimand = model.identify_effect()

# Estimate the causal effect using a linear regression estimator suitable for continuous outcomes
causal_estimate = model.estimate_effect(
    identified_estimand,
    method_name="backdoor.linear_regression",
    control_value=0,
    treatment_value=1
)


print("Identified Estimand:", identified_estimand)
print("Causal Estimate:", causal_estimate)

In [ ]:
# IPW Model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from joblib import Parallel, delayed


df = pd.read_stata('SIPP_Paid_Leave.dta')


df['ssuid'] = df['ssuid'].astype(str)
df['spanel'] = df['spanel'].astype(int)
df['epppnum'] = df['epppnum'].astype(int)
df['tfipsst'] = df['tfipsst'].astype('int8')
df['tage'] = df['tage'].astype('int8')
df['eeducate'] = df['eeducate'].astype('int8')
df['rmesr'] = df['rmesr'].astype('int8')
df['esex'] = df['esex'].astype('int8')
df['wpfinwgt'] = df['wpfinwgt'].astype(float)


def categorize_education(edu_code):
    if edu_code in range(32, 39):
        return 'Less than High School'
    elif edu_code == 39:
        return 'High School Graduate'
    elif edu_code in range(40, 42):
        return 'Some College'
    elif edu_code in range(42, 44):
        return 'Associate Degree'
    elif edu_code == 44:
        return "Bachelor's Degree"
    elif edu_code in range(45, 48):
        return 'Graduate Degree'
    else:
        return np.nan

df['education_category'] = df['eeducate'].apply(categorize_education)


df = df.dropna(subset=['education_category', 'rmesr'])


df['treated'] = ((df['tfipsst'] == 6) | (df['tfipsst'] == 34)).astype(int)
df['rm_lfp'] = (df['rmesr'] <= 7).astype(int)


df = pd.get_dummies(df, columns=['education_category'], drop_first=True)


T = 'treated'
Y = 'rm_lfp'
covariates = ['tage', 'esex'] + [col for col in df.columns if col.startswith('education_category_')]


scaler = StandardScaler()
df[covariates] = scaler.fit_transform(df[covariates])


ps_model = LogisticRegression(max_iter=1000, random_state=42)
ps_model.fit(df[covariates], df[T])
df['propensity_score'] = ps_model.predict_proba(df[covariates])[:, 1]


weights = np.where(df[T] == 1, 1 / df['propensity_score'], 1 / (1 - df['propensity_score']))


def compute_ate_ipw(df, weights, T, Y):
    treated_outcome = np.sum(weights[df[T] == 1] * df.loc[df[T] == 1, Y]) / np.sum(weights[df[T] == 1])
    control_outcome = np.sum(weights[df[T] == 0] * df.loc[df[T] == 0, Y]) / np.sum(weights[df[T] == 0])
    return treated_outcome - control_outcome

ate = compute_ate_ipw(df, weights, T, Y)
print("ATE:", ate)

# Bootstrap confidence intervals for ATE
np.random.seed(42)
bootstrap_samples = 1000
ates = []
for _ in range(bootstrap_samples):
    bootstrap_sample = df.sample(frac=1, replace=True)
    bootstrap_weights = np.where(
        bootstrap_sample[T] == 1, 
        1 / bootstrap_sample['propensity_score'], 
        1 / (1 - bootstrap_sample['propensity_score'])
    )
    ates.append(compute_ate_ipw(bootstrap_sample, bootstrap_weights, T, Y))

ci_lower = np.percentile(ates, 2.5)
ci_upper = np.percentile(ates, 97.5)
print(f"95% CI for ATE: ({ci_lower}, {ci_upper})")


plt.figure(figsize=(10, 6))
sns.histplot(
    df[df[T] == 0]['propensity_score'], 
    label='Control', 
    color='#FFD966',  # Mustard yellow
    kde=True, 
    bins=20, 
    edgecolor=None  # Remove black edges
)
sns.histplot(
    df[df[T] == 1]['propensity_score'], 
    label='Treated', 
    color='#B2F699',  # Pastel green
    kde=True, 
    bins=20, 
    edgecolor=None  # Remove black edges
)
plt.title("Positivity Check: IPW Model", fontsize=16)
plt.xlabel("Propensity Score", fontsize=14)
plt.ylabel("Density", fontsize=14)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

# Plot ATE bootstrap distribution
plt.figure(figsize=(10, 6))
sns.histplot(ates, kde=True, bins=30, color='#88CCEE', edgecolor=None)
plt.axvline(ate, color='blue', linestyle='-', linewidth=2, label=f'ATE: {ate:.4f}')
plt.axvline(ci_lower, color='red', linestyle='--', linewidth=2, label=f'2.5%: {ci_lower:.4f}')
plt.axvline(ci_upper, color='red', linestyle='--', linewidth=2, label=f'97.5%: {ci_upper:.4f}')
plt.title("ATE Bootstrap Distribution", fontsize=16)
plt.xlabel("ATE", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Matching Methods

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors


df = pd.read_stata('SIPP_Paid_Leave.dta')


df['ssuid'] = df['ssuid'].astype(str)
df['spanel'] = df['spanel'].astype(int)
df['epppnum'] = df['epppnum'].astype(int)
df['tfipsst'] = df['tfipsst'].astype('int8')
df['tage'] = df['tage'].astype('int8')
df['eeducate'] = df['eeducate'].astype('int8')
df['rmesr'] = df['rmesr'].astype('int8')
df['esex'] = df['esex'].astype('int8')
df['wpfinwgt'] = df['wpfinwgt'].astype(float)


def categorize_education(edu_code):
    if edu_code in range(32, 39):
        return 'Less than High School'
    elif edu_code == 39:
        return 'High School Graduate'
    elif edu_code in range(40, 42):
        return 'Some College'
    elif edu_code in range(42, 44):
        return 'Associate Degree'
    elif edu_code == 44:
        return "Bachelor's Degree"
    elif edu_code in range(45, 48):
        return 'Graduate Degree'
    else:
        return np.nan

df['education_category'] = df['eeducate'].apply(categorize_education)


df = df.dropna(subset=['education_category', 'rmesr'])

df['treated'] = ((df['tfipsst'] == 6) | (df['tfipsst'] == 34)).astype(int)
df['rm_lfp'] = (df['rmesr'] <= 7).astype(int)


df = pd.get_dummies(df, columns=['education_category'], drop_first=True)


T = 'treated'
Y = 'rm_lfp'
covariates = ['tage', 'esex'] + [col for col in df.columns if col.startswith('education_category_')]


scaler = StandardScaler()
df[covariates] = scaler.fit_transform(df[covariates])


ps_model = LogisticRegression(max_iter=1000, random_state=42)
ps_model.fit(df[covariates], df[T])
df['propensity_score'] = ps_model.predict_proba(df[covariates])[:, 1]


nn = NearestNeighbors(n_neighbors=1, metric='euclidean')
control_indices = df[df[T] == 0].index
treated_indices = df[df[T] == 1].index
nn.fit(df.loc[control_indices, ['propensity_score']])
distances, indices = nn.kneighbors(df.loc[treated_indices, ['propensity_score']])
matched_indices = control_indices[indices.flatten()]


matched_data = pd.concat([
    df.loc[treated_indices],
    df.loc[matched_indices]
])

# Balance table function
def balance_table(data, covariates, treatment):
    balance = pd.DataFrame(columns=['Covariate', 'Mean Treated', 'Mean Control', 'Standardized Difference'])
    for cov in covariates:
        mean_treated = data[data[treatment] == 1][cov].mean()
        mean_control = data[data[treatment] == 0][cov].mean()
        std_diff = (mean_treated - mean_control) / data[cov].std()
        balance = pd.concat([
            balance,
            pd.DataFrame({
                'Covariate': [cov],
                'Mean Treated': [mean_treated],
                'Mean Control': [mean_control],
                'Standardized Difference': [std_diff]
            })
        ], ignore_index=True)
    return balance

# Balance tables
balance_pre = balance_table(df, covariates, T)
balance_post = balance_table(matched_data, covariates, T)


y_treated = matched_data[matched_data[T] == 1][Y].mean()
y_control = matched_data[matched_data[T] == 0][Y].mean()
ate = y_treated - y_control

print("ATE:", ate)


plt.figure(figsize=(10, 6))
sns.histplot(
    df[df['treated'] == 0]['propensity_score'], 
    label='Control', 
    color='#AEC6CF',  # Pastel blue
    kde=True, 
    bins=20, 
    edgecolor=None  # Remove black boundaries
)
sns.histplot(
    df[df['treated'] == 1]['propensity_score'], 
    label='Treated', 
    color='#FFB6C1',  # Pastel pink
    kde=True, 
    bins=20, 
    edgecolor=None  # Remove black boundaries
)
plt.title("Propensity Score Distribution", fontsize=16)
plt.xlabel("Propensity Score", fontsize=14)
plt.ylabel("Density", fontsize=14)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()
# Display balance tables
print("Pre-Matching Balance Table")
print(balance_pre)
print("\nPost-Matching Balance Table")
print(balance_post)


np.random.seed(42)
bootstrap_ates = []
for _ in range(500):
    boot_sample = matched_data.sample(frac=1, replace=True)
    y_treated_boot = boot_sample[boot_sample[T] == 1][Y].mean()
    y_control_boot = boot_sample[boot_sample[T] == 0][Y].mean()
    bootstrap_ates.append(y_treated_boot - y_control_boot)

# Calculate confidence intervals
ci_lower = np.percentile(bootstrap_ates, 2.5)
ci_upper = np.percentile(bootstrap_ates, 97.5)


print(f"ATE: {ate}")
print(f"95% Confidence Interval: [{ci_lower}, {ci_upper}]")


plt.figure(figsize=(10, 6))
sns.histplot(bootstrap_ates, kde=True, color='#88CCEE', bins=30, edgecolor=None)  # Removed black boundaries
plt.axvline(ci_lower, color='red', linestyle='--', label=f"2.5%: {ci_lower:.4f}")
plt.axvline(ci_upper, color='red', linestyle='--', label=f"97.5%: {ci_upper:.4f}")
plt.axvline(ate, color='blue', linestyle='-', label=f"ATE: {ate:.4f}")
plt.title("ATE Bootstrap Distribution", fontsize=16)
plt.xlabel("ATE", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Doubly Robust Model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed


df = pd.read_stata('SIPP_Paid_Leave.dta')


df['ssuid'] = df['ssuid'].astype(str)
df['spanel'] = df['spanel'].astype(int)
df['epppnum'] = df['epppnum'].astype(int)
df['tfipsst'] = df['tfipsst'].astype('int8')
df['tage'] = df['tage'].astype('int8')
df['eeducate'] = df['eeducate'].astype('int8')
df['rmesr'] = df['rmesr'].astype('int8')
df['esex'] = df['esex'].astype('int8')
df['wpfinwgt'] = df['wpfinwgt'].astype(float)


def categorize_education(edu_code):
    if edu_code in range(32, 39):
        return 'Less than High School'
    elif edu_code == 39:
        return 'High School Graduate'
    elif edu_code in range(40, 42):
        return 'Some College'
    elif edu_code in range(42, 44):
        return 'Associate Degree'
    elif edu_code == 44:
        return "Bachelor's Degree"
    elif edu_code in range(45, 48):
        return 'Graduate Degree'
    else:
        return np.nan

df['education_category'] = df['eeducate'].apply(categorize_education)


df = df.dropna(subset=['education_category', 'rmesr'])


df['treated'] = ((df['tfipsst'] == 6) | (df['tfipsst'] == 34)).astype(int)
df['rm_lfp'] = (df['rmesr'] <= 7).astype(int)


df = pd.get_dummies(df, columns=['education_category'], drop_first=False)


T = 'treated'
Y = 'rm_lfp'
covariates = ['tage', 'esex'] + [col for col in df.columns if col.startswith('education_category_')]


scaler = StandardScaler()
df[covariates] = scaler.fit_transform(df[covariates])


ps_model = LogisticRegression(max_iter=1000, random_state=42)
ps_model.fit(df[covariates], df[T])
df['propensity_score'] = ps_model.predict_proba(df[covariates])[:, 1]


def doubly_robust(df, X, T, Y):
    ps = LogisticRegression(C=1e6, max_iter=1000).fit(df[X], df[T]).predict_proba(df[X])[:, 1]
    mu0 = LinearRegression().fit(df.query(f"{T}==0")[X], df.query(f"{T}==0")[Y]).predict(df[X])
    mu1 = LinearRegression().fit(df.query(f"{T}==1")[X], df.query(f"{T}==1")[Y]).predict(df[X])
    return (
        np.mean(df[T] * (df[Y] - mu1) / ps + mu1) -
        np.mean((1 - df[T]) * (df[Y] - mu0) / (1 - ps) + mu0)
    )


doubly_robust_ate = doubly_robust(df, covariates, T, Y)
print("Double Robust Average Treatment Effect (ATE):", doubly_robust_ate)


np.random.seed(88)
bootstrap_samples = 20
ates = Parallel(n_jobs=4)(delayed(doubly_robust)(df.sample(frac=1, replace=True), covariates, T, Y) for _ in range(bootstrap_samples))
ates = np.array(ates)

ci_lower = np.percentile(ates, 2.5)
ci_upper = np.percentile(ates, 97.5)
print(f"95% CI: ({ci_lower}, {ci_upper})")

plt.figure(figsize=(10, 6))
sns.histplot(ates, kde=True, bins=30, color='#88CCEE', edgecolor=None)
plt.axvline(doubly_robust_ate, color='blue', linestyle='-', linewidth=2, label=f'ATE: {doubly_robust_ate:.4f}')
plt.axvline(ci_lower, color='red', linestyle='--', linewidth=2, label=f'2.5%: {ci_lower:.4f}')
plt.axvline(ci_upper, color='red', linestyle='--', linewidth=2, label=f'97.5%: {ci_upper:.4f}')
plt.title("ATE Bootstrap Distribution", fontsize=16)
plt.xlabel("ATE", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

# Positivity check for education levels
education_labels = ['Less than High School', 'High School Graduate', 'Some College', 'Associate Degree', "Bachelor's Degree", 'Graduate Degree']
education_columns = [col for col in df.columns if col.startswith('education_category_')]

df['education_category'] = df[education_columns].idxmax(axis=1).str.replace('education_category_', '')

plt.figure(figsize=(12, 8))
sns.boxplot(data=df, x='education_category', y='propensity_score', hue='treated', palette=['#FFCC99', '#C3E6CB'])
plt.xticks(ticks=range(len(education_labels)), labels=education_labels, rotation=45, ha='right')
plt.title("Positivity Check by Education Levels", fontsize=16)
plt.xlabel("Education Category", fontsize=14)
plt.ylabel("Propensity Score", fontsize=14)
plt.legend(title='Group', labels=['Control', 'Treated'], fontsize=12, title_fontsize=12)
plt.tight_layout()
plt.show()

# Positivity check for age
plt.figure(figsize=(12, 8))
sns.histplot(
    data=df[df['treated'] == 0], 
    x='tage', 
    label='Control', 
    color='#FFCC99', 
    bins=20, 
    alpha=0.7, 
    stat='density', 
    edgecolor=None
)
sns.histplot(
    data=df[df['treated'] == 1], 
    x='tage', 
    label='Treated', 
    color='#C3E6CB', 
    bins=20, 
    alpha=0.7, 
    stat='density', 
    edgecolor=None
)
plt.title("Positivity Check by Age", fontsize=16)
plt.xlabel("Age", fontsize=14)
plt.ylabel("Density", fontsize=14)
plt.legend(title='Group', fontsize=12, title_fontsize=12)
plt.tight_layout()
plt.show()



In [ ]:
# DR Learner

!pip install econml -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LassoCV
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from econml.dr import LinearDRLearner


df = pd.read_stata('SIPP_Paid_Leave.dta')


df['ssuid'] = df['ssuid'].astype(str)
df['spanel'] = df['spanel'].astype(int)
df['epppnum'] = df['epppnum'].astype(int)
df['tfipsst'] = df['tfipsst'].astype('int8')
df['srefmon'] = df['srefmon'].astype('int8')  # Reference month
df['tage'] = df['tage'].astype('int8')  # Age of respondent
df['eeducate'] = df['eeducate'].astype('int8')  # Education level
df['rmesr'] = df['rmesr'].astype('int8')
df['esex'] = df['esex'].astype('int8')  # Sex of respondent


df.sort_values(by=['ssuid', 'epppnum', 'spanel', 'swave', 'srefmon'], inplace=True)
df['sippid'] = df.groupby(['spanel', 'ssuid', 'epppnum']).ngroup() + 1
df['months'] = df.groupby('sippid').cumcount() + 1


df['birth_month'] = pd.to_datetime(df['birth_month'], format='%Y-%m')
df['date'] = pd.to_datetime(dict(year=df['rhcalyr'], month=df['srefmon'], day=1))
df['birth'] = (df['date'] - df['birth_month']).apply(lambda x: x.days // 30 if not pd.isna(x) else np.nan).astype(float)
df['birth_seen_f'] = (df['birth'] == 0).astype(int)
df['birth_seen'] = df.groupby('sippid')['birth_seen_f'].transform('max').astype(int)

def min_birth(x):
    cond = (x['birth'] > 0) & (x['birth_seen'] == 0)
    return x.loc[cond, 'birth'].min() if cond.any() else np.nan

df['ref_month_ns'] = df.groupby('sippid').apply(min_birth).reset_index(level=0, drop=True).astype(float)
df['ref_month'] = np.nan
df.loc[(df['birth'] == 0) & (df['birth_seen'] == 1), 'ref_month'] = 1
df.loc[(df['ref_month_ns'] == df['birth']) & (df['birth_seen'] == 0), 'ref_month'] = 1
df['ref_month'] = df['ref_month'].astype(float)

es
state_labels = {6: "California", 34: "New Jersey", 12: "Florida", 48: "Texas", 36: "New York"}
df['state'] = df['tfipsst'].map(state_labels).astype('category')
df['CA_date'] = pd.to_datetime('2004-07-01')
df['NJ_date'] = pd.to_datetime('2009-07-01')
df['post_policy'] = 0
df.loc[(df['tfipsst'] == 34) & (df['birth_month'] >= df['NJ_date']), 'post_policy'] = 1
df.loc[(df['tfipsst'] == 6) & (df['birth_month'] >= df['CA_date']), 'post_policy'] = 1


df['rm_lfp'] = (df['rmesr'] <= 7).astype(int)
df.loc[df['rmesr'] == -1, 'rm_lfp'] = np.nan
df['treated'] = ((df['state'].isin(['California', 'New Jersey'])) & (df['post_policy'] == 1)).astype(int)


df = df.dropna(subset=['rm_lfp', 'treated', 'state', 'post_policy'])


columns_to_drop = ['birth_month', 'date', 'CA_date', 'NJ_date', 'end_date']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])


categ = ["state", "esex", "eeducate"]
data_with_categ = pd.concat([
    df.drop(columns=categ),  
    pd.get_dummies(df[categ], drop_first=True)  
], axis=1)

imputer = SimpleImputer(strategy='mean')
data_with_categ_imputed = pd.DataFrame(imputer.fit_transform(data_with_categ), columns=data_with_categ.columns)


y = data_with_categ_imputed['rm_lfp'].values  # Outcome: Labor force participation
T = data_with_categ_imputed['treated'].values  # Treatment indicator


X = data_with_categ_imputed[
    ['srefmon', 'treated', 'post_policy', 'tage', 'eeducate']
].rename(columns={
    'srefmon': 'Reference Month',
    'treated': 'Treated Indicator',
    'post_policy': 'Post Policy',
    'tage': 'Age of Respondent',
    'eeducate': 'Level of Education'
}).values


est = LinearDRLearner(
    model_regression=LassoCV(cv=3),
    model_propensity=DummyClassifier(strategy='prior')
)


est.fit(y, T, X=X)

# Estimate the treatment effect for a specific segment
effect = est.effect(np.array([[6, 1, 1, 35, 2]]))  # Example segment
print("Estimated Treatment Effect for segment:", effect)

# Calculate 95% Confidence Interval for the effect
effect_ci = est.effect_interval(np.array([[6, 1, 1, 35, 2]]), alpha=.05)
print("95% Confidence Interval for Treatment Effect:", effect_ci)

# Detailed inference including point estimate, standard error, z score, p value, and confidence interval
effect_inference = est.effect_inference(np.array([[6, 1, 1, 35, 2]])).summary_frame(alpha=.05)
print("Effect Inference Summary:\n", effect_inference)


feature_names = [
    'Reference Month', 'Treated Indicator', 'Post Policy', 
    'Age of Respondent', 'Level of Education'
]
coef_with_names = np.array(list(zip(est.cate_feature_names(feature_names), est.coef_(T=1))))
print("Coefficients with Feature Names:\n", coef_with_names)
print("Intercept for CATE model:", est.intercept_(T=1))


lower, upper = np.array(est.coef__interval(T=1))
point_estimate = est.coef_(T=1)
yerr = np.zeros((2, point_estimate.shape[0]))
yerr[0, :] = point_estimate - lower
yerr[1, :] = upper - point_estimate


with sns.axes_style("darkgrid"):
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    x = np.arange(len(point_estimate))
    plt.errorbar(x, point_estimate, yerr=yerr, fmt='o', capsize=5)
    ax.set_xticks(x)
    ax.set_xticklabels(feature_names, rotation='vertical', fontsize=12)
    ax.set_ylabel('Coefficient')
    plt.title("Coefficient Estimates with 95% Confidence Intervals")
    plt.show()


In [ ]:
!pip install econml

In [ ]:
# Double Machine Learning

import pandas as pd
import numpy as np
from econml.dml import LinearDML
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split

df = pd.read_stata('SIPP_Paid_Leave.dta')


df['ssuid'] = df['ssuid'].astype(str)
df['spanel'] = df['spanel'].astype(int)
df['epppnum'] = df['epppnum'].astype(int)
df['tfipsst'] = df['tfipsst'].astype('int8')
df['rhcalyr'] = df['rhcalyr'].astype(int)
df['swave'] = df['swave'].astype('int8')
df['srefmon'] = df['srefmon'].astype('int8')
df['rhcalmn'] = df['rhcalmn'].astype('int8')
df['tage'] = df['tage'].astype('int8')
df['eeducate'] = df['eeducate'].astype('int8')
df['rmesr'] = df['rmesr'].astype('int8')
df['esex'] = df['esex'].astype('int8')
df['wpfinwgt'] = df['wpfinwgt'].astype(float)


df.sort_values(by=['ssuid', 'epppnum', 'spanel', 'swave', 'srefmon'], inplace=True)
df['sippid'] = df.groupby(['spanel', 'ssuid', 'epppnum']).ngroup() + 1


df['months'] = df.groupby('sippid').cumcount() + 1


df['birth_month'] = pd.to_datetime(df['birth_month'], format='%Y-%m').dt.to_period('M')
df['date'] = pd.to_datetime(dict(year=df['rhcalyr'], month=df['rhcalmn'], day=1)).dt.to_period('M')


df['birth'] = (df['date'] - df['birth_month']).apply(lambda x: x.n if not pd.isna(x) else np.nan)
df['birth'] = df['birth'].astype(float)


df['rm_lfp'] = (df['rmesr'] <= 7).astype(int)
df.loc[df['rmesr'] == -1, 'rm_lfp'] = np.nan


df['CA_date'] = pd.Period('2004-07', freq='M')
df['NJ_date'] = pd.Period('2009-07', freq='M')
df['post_policy'] = 0
df.loc[(df['tfipsst'] == 34) & (df['birth_month'] >= df['NJ_date']), 'post_policy'] = 1
df.loc[(df['tfipsst'] == 6) & (df['birth_month'] >= df['CA_date']), 'post_policy'] = 1


df['treated'] = ((df['tfipsst'].isin([6, 34]))).astype(int)
df['treated_post'] = df['treated'] * df['post_policy']  # Interaction term (DiD)


df = df.dropna(subset=['rm_lfp', 'treated', 'post_policy', 'treated_post', 'tage', 'eeducate', 'esex', 'wpfinwgt', 'rhcalyr'])


Y = df['rm_lfp']  # Outcome
T = df['treated_post']  # Treatment
X = df[['tage', 'eeducate']]  # Covariates
W = df[['wpfinwgt', 'rhcalyr']]  # Controls


X_train, X_test, Y_train, Y_test, T_train, T_test, W_train, W_test = train_test_split(
    X, Y, T, W, test_size=0.2, random_state=42
)


dml_model = LinearDML(
    model_y=RandomForestRegressor(),
    model_t=RandomForestClassifier(min_samples_leaf=10),
    discrete_treatment=True,
    cv=5  # Cross-validation folds
)


dml_model.fit(Y_train, T_train, X=X_train, W=W_train)


te_pred = dml_model.effect(X_test)
lb, ub = dml_model.effect_interval(X_test, alpha=0.05)


print("Estimated Treatment Effects:")
print(te_pred)

print("\n95% Confidence Intervals:")
for i in range(len(lb)):
    print(f"[{lb[i]:.3f}, {ub[i]:.3f}]")


ate = dml_model.ate(X_test)
ate_lb, ate_ub = dml_model.ate_interval(X_test, alpha=0.05)
print(f"\nAverage Treatment Effect: {ate:.3f}")
print(f"95% CI for ATE: [{ate_lb:.3f}, {ate_ub:.3f}]")


In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Sort the treatment effects for cumulative distribution
sorted_te = np.sort(te_pred)

# Generate cumulative probabilities
cumulative_probs = np.linspace(0, 1, len(sorted_te))

# Plot cumulative distribution
plt.figure(figsize=(10, 6))
sns.lineplot(x=sorted_te, y=cumulative_probs, color='#88CCEE', linewidth=2)
plt.axvline(0, color='gray', linestyle='--', linewidth=1, label="Zero Effect")
plt.title("Cumulative Distribution of Treatment Effects", fontsize=16)
plt.xlabel("Estimated Treatment Effect", fontsize=14)
plt.ylabel("Cumulative Probability", fontsize=14)
plt.grid(alpha=0.3)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()



In [ ]:
# Ensure X_test is numeric
if isinstance(X_test, pd.DataFrame):
    X_test = X_test.to_numpy()  # Convert DataFrame to NumPy array
X_test = X_test.astype(float)  # Ensure all values are numeric


expected_te1 = np.array([x_i[0] for x_i in X_test])  # True linear effect
expected_te2 = np.array([x_i[0]**2 for x_i in X_test]).flatten()  # True quadratic effect


plt.figure(figsize=(15, 6))


plt.subplot(1, 2, 1)
plt.title("LinearDML with 2D-Poly Features")
plt.plot(X_test[:, 0], te_pred, label='DML estimate', color='blue', alpha=0.6)
plt.fill_between(X_test[:, 0], lb, ub, color='blue', alpha=0.3, label='Confidence Interval')
plt.plot(X_test[:, 0], expected_te1, '--', color='orange', label='True Effect 1')
plt.plot(X_test[:, 0], expected_te2, '--', color='green', label='True Effect 2')
plt.ylabel("Treatment Effect")
plt.xlabel("x (First Feature of X_test)")
plt.legend()
plt.grid()


plt.subplot(1, 2, 2)
plt.title("SparseLinearDML with 2D-Poly Features")
plt.plot(X_test[:, 0], te_pred2, label='DML estimate', color='purple', alpha=0.6)
plt.fill_between(X_test[:, 0], lb2, ub2, color='purple', alpha=0.3, label='Confidence Interval')
plt.plot(X_test[:, 0], expected_te1, '--', color='orange', label='True Effect 1')
plt.plot(X_test[:, 0], expected_te2, '--', color='green', label='True Effect 2')
plt.ylabel("Treatment Effect")
plt.xlabel("x (First Feature of X_test)")
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


In [ ]:
!pip install econml

In [ ]:
# Casual Forest Model

import pandas as pd
import numpy as np
from econml.grf import CausalForest
import matplotlib.pyplot as plt


df = pd.read_stata('SIPP_Paid_Leave.dta')


df['ssuid'] = df['ssuid'].astype(str)
df['spanel'] = df['spanel'].astype(int)
df['epppnum'] = df['epppnum'].astype(int)
df['tfipsst'] = df['tfipsst'].astype('int8')
df['rhcalyr'] = df['rhcalyr'].astype(int)
df['swave'] = df['swave'].astype('int8')
df['srefmon'] = df['srefmon'].astype('int8')
df['rhcalmn'] = df['rhcalmn'].astype('int8')
df['tage'] = df['tage'].astype('int8')
df['eeducate'] = df['eeducate'].astype('int8')
df['rmesr'] = df['rmesr'].astype('int8')
df['esex'] = df['esex'].astype('int8')
df['wpfinwgt'] = df['wpfinwgt'].astype(float)


df.sort_values(by=['ssuid', 'epppnum', 'spanel', 'swave', 'srefmon'], inplace=True)
df['sippid'] = df.groupby(['spanel', 'ssuid', 'epppnum']).ngroup() + 1


df['months'] = df.groupby('sippid').cumcount() + 1


df['birth_month'] = pd.to_datetime(df['birth_month'], format='%Y-%m').dt.to_period('M')
df['date'] = pd.to_datetime(dict(year=df['rhcalyr'], month=df['rhcalmn'], day=1)).dt.to_period('M')


df['birth'] = (df['date'] - df['birth_month']).apply(lambda x: x.n if not pd.isna(x) else np.nan)
df['birth'] = df['birth'].astype(float)


df['rm_lfp'] = (df['rmesr'] <= 7).astype(int)
df.loc[df['rmesr'] == -1, 'rm_lfp'] = np.nan


df['CA_date'] = pd.Period('2004-07', freq='M')
df['NJ_date'] = pd.Period('2009-07', freq='M')
df['post_policy'] = 0
df.loc[(df['tfipsst'] == 34) & (df['birth_month'] >= df['NJ_date']), 'post_policy'] = 1
df.loc[(df['tfipsst'] == 6) & (df['birth_month'] >= df['CA_date']), 'post_policy'] = 1


df['treated'] = ((df['tfipsst'].isin([6, 34]))).astype(int)
df['treated_post'] = df['treated'] * df['post_policy']  # Interaction term (DiD)


df = df.dropna(subset=['rm_lfp', 'treated', 'post_policy', 'treated_post', 'tage', 'eeducate', 'esex', 'wpfinwgt', 'rhcalyr'])


Y = df['rm_lfp'].values.reshape(-1, 1)  # Outcome
T = df['treated_post'].values.reshape(-1, 1)  # Treatment
X = df[['tage', 'eeducate']].values  # Covariates


n_samples = X.shape[0]
X_test = X[:min(100, n_samples)].copy()
X_test[:, 0] = np.linspace(np.percentile(X[:, 0], 1), np.percentile(X[:, 0], 99), min(100, n_samples))

# Initialize and fit the CausalForest model
est = CausalForest(
    criterion='het',
    n_estimators=400,
    min_samples_leaf=5,
    max_depth=None,
    min_var_leaf_on_val=True,
    max_samples=0.45,
    min_balancedness_tol=0.45,
    honest=True,
    verbose=0,
    n_jobs=-1,
    random_state=1235
)

est.fit(X, T, Y)


point, lb, ub = est.predict(X_test, interval=True, alpha=0.01)

# Highlight specific bins corresponding to subgroups in the histogram
# Example: Highlight bins for treatment effects within specific ranges (-0.05 to 0 for 'Low Effect', 0.05 to 0.1 for 'High Effect')

plt.figure(figsize=(10, 6))


counts, bins, patches = plt.hist(
    point, bins=30, alpha=0.7, edgecolor=None, color='#88CCEE', linewidth=0
)  # Adjusted color and removed gaps between bars


for i in range(len(bins) - 1):
    if bins[i] >= -0.05 and bins[i + 1] <= 0:
        patches[i].set_facecolor('#FFCC99')  # Highlight low effects in orange
        patches[i].set_label('Low Effect')  # Add label for legend
    elif bins[i] >= 0.05 and bins[i + 1] <= 0.1:
        patches[i].set_facecolor('#C3E6CB')  # Highlight high effects in green
        patches[i].set_label('High Effect')  # Add label for legend


plt.title("Histogram of Heterogeneous Treatment Effects", fontsize=16)
plt.xlabel("Estimated Treatment Effect", fontsize=14)
plt.ylabel("Frequency", fontsize=14)
plt.axvline(x=0, color='gray', linestyle='--', label="Zero Effect")  # Add zero effect line
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), fontsize=12)  # Ensure unique labels in the legend
plt.tight_layout()
plt.show()

print("Point Estimates of Heterogeneous Treatment Effects:")
print(point)

# Print results
print("Point Estimates of Treatment Effects:", point)
print("\n99% Confidence Intervals:")
for i in range(len(lb)):
    print(f"[{lb[i]:.3f}, {ub[i]:.3f}]")



In [ ]:
# Causal Forest Model

import pandas as pd
import numpy as np
from econml.dml import CausalForestDML
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


df = pd.read_stata('SIPP_Paid_Leave.dta')


df['ssuid'] = df['ssuid'].astype(str)
df['spanel'] = df['spanel'].astype(int)
df['epppnum'] = df['epppnum'].astype(int)
df['tfipsst'] = df['tfipsst'].astype('int8')
df['rhcalyr'] = df['rhcalyr'].astype(int)
df['swave'] = df['swave'].astype('int8')
df['srefmon'] = df['srefmon'].astype('int8')
df['rhcalmn'] = df['rhcalmn'].astype('int8')
df['tage'] = df['tage'].astype('int8')
df['eeducate'] = df['eeducate'].astype('int8')
df['rmesr'] = df['rmesr'].astype('int8')
df['esex'] = df['esex'].astype('int8')
df['wpfinwgt'] = df['wpfinwgt'].astype(float)


df.sort_values(by=['ssuid', 'epppnum', 'spanel', 'swave', 'srefmon'], inplace=True)
df['sippid'] = df.groupby(['spanel', 'ssuid', 'epppnum']).ngroup() + 1

df['months'] = df.groupby('sippid').cumcount() + 1


df['birth_month'] = pd.to_datetime(df['birth_month'], format='%Y-%m').dt.to_period('M')
df['date'] = pd.to_datetime(dict(year=df['rhcalyr'], month=df['rhcalmn'], day=1)).dt.to_period('M')


df['birth'] = (df['date'] - df['birth_month']).apply(lambda x: x.n if not pd.isna(x) else np.nan)
df['birth'] = df['birth'].astype(float)


df['rm_lfp'] = (df['rmesr'] <= 7).astype(int)
df.loc[df['rmesr'] == -1, 'rm_lfp'] = np.nan


df['CA_date'] = pd.Period('2004-07', freq='M')
df['NJ_date'] = pd.Period('2009-07', freq='M')
df['post_policy'] = 0
df.loc[(df['tfipsst'] == 34) & (df['birth_month'] >= df['NJ_date']), 'post_policy'] = 1
df.loc[(df['tfipsst'] == 6) & (df['birth_month'] >= df['CA_date']), 'post_policy'] = 1


df['treated'] = ((df['tfipsst'].isin([6, 34]))).astype(int)
df['treated_post'] = df['treated'] * df['post_policy']  # Interaction term (DiD)


df = df.dropna(subset=['rm_lfp', 'treated', 'post_policy', 'treated_post', 'tage', 'eeducate', 'esex', 'wpfinwgt', 'rhcalyr'])


Y = df['rm_lfp']  # Outcome
T = df['treated_post']  # Treatment
X = df[['tage', 'eeducate']]  # Covariates
W = df[['wpfinwgt', 'rhcalyr']]  # Controls


X_train, X_test, Y_train, Y_test, T_train, T_test, W_train, W_test = train_test_split(
    X, Y, T, W, test_size=0.2, random_state=42
)


causal_forest_model = CausalForestDML(
    model_y=RandomForestRegressor(),
    model_t=RandomForestRegressor(),  # Use regressor as treatment model
    n_estimators=50,  # Number of estimators
    min_samples_leaf=5,
    max_depth=10,
    subforest_size=5,  # Ensure divisibility
    random_state=42
)


causal_forest_model.fit(Y_train, T_train, X=X_train, W=W_train)


te_pred = causal_forest_model.effect(X_test)
te_lower, te_upper = causal_forest_model.effect_interval(X_test, alpha=0.05)


plt.figure(figsize=(12, 6))
plt.plot(range(len(te_pred)), te_pred, label='Estimated Treatment Effect', color='blue')
plt.fill_between(
    range(len(te_pred)), te_lower, te_upper, color='lightblue', alpha=0.5, label='95% CI'
)
plt.axhline(y=0, color='gray', linestyle='--', label='Zero Effect')
plt.title('Estimated Treatment Effects with Causal Forest DML', fontsize=16)
plt.xlabel('Sample Index', fontsize=14)
plt.ylabel('Estimated Treatment Effect', fontsize=14)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

# Optional: Evaluate average treatment effect
ate = causal_forest_model.ate(X_test)
ate_lb, ate_ub = causal_forest_model.ate_interval(X_test, alpha=0.05)
print(f"\nAverage Treatment Effect: {ate:.3f}")
print(f"95% CI for ATE: [{ate_lb:.3f}, {ate_ub:.3f}]")


In [ ]:
# Meta Learner Models

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression


df = pd.read_stata('SIPP_Paid_Leave.dta')


df['treated_post'] = df['treated'] * df['post_policy']


X = ['tage', 'eeducate']  # Covariates
T = 'treated_post'  # Treatment
Y = 'rm_lfp'  # Outcome


education_categories = {
    1: 'Less than High School',
    2: 'High School Graduate',
    3: 'Some College',
    4: 'Associate Degree',
    5: "Bachelor's Degree",
    6: 'Graduate Degree'
}

age_categories = {
    '24-35': (24, 35),
    '36-45': (36, 45)
}


train, test = train_test_split(df, test_size=0.2, random_state=42)

# S-Learner
s_learner = LGBMRegressor(max_depth=3, min_child_samples=30)
s_learner.fit(train[X + [T]], train[Y])

s_cate_education = {}
s_cate_age = {}

for category, label in education_categories.items():
    subgroup = test[test['eeducate'] == category]
    s_cate_education[label] = (
        s_learner.predict(subgroup[X].assign(**{T: 1})) -
        s_learner.predict(subgroup[X].assign(**{T: 0}))
    ).mean()

for label, (min_age, max_age) in age_categories.items():
    subgroup = test[(test['tage'] >= min_age) & (test['tage'] <= max_age)]
    s_cate_age[label] = (
        s_learner.predict(subgroup[X].assign(**{T: 1})) -
        s_learner.predict(subgroup[X].assign(**{T: 0}))
    ).mean()

# T-Learner
m0 = LGBMRegressor(max_depth=2, min_child_samples=60)
m1 = LGBMRegressor(max_depth=2, min_child_samples=60)

m0.fit(train.query(f"{T}==0")[X], train.query(f"{T}==0")[Y])
m1.fit(train.query(f"{T}==1")[X], train.query(f"{T}==1")[Y])

t_cate_education = {}
t_cate_age = {}

for category, label in education_categories.items():
    subgroup = test[test['eeducate'] == category]
    t_cate_education[label] = (
        m1.predict(subgroup[X]) - m0.predict(subgroup[X])
    ).mean()

for label, (min_age, max_age) in age_categories.items():
    subgroup = test[(test['tage'] >= min_age) & (test['tage'] <= max_age)]
    t_cate_age[label] = (
        m1.predict(subgroup[X]) - m0.predict(subgroup[X])
    ).mean()

# X-Learner
x_learner = {
    'm0': LGBMRegressor(max_depth=3, min_child_samples=30),
    'm1': LGBMRegressor(max_depth=3, min_child_samples=30),
    'tau0': RandomForestRegressor(n_estimators=100, max_depth=3, min_samples_leaf=10),
    'tau1': RandomForestRegressor(n_estimators=100, max_depth=3, min_samples_leaf=10)
}

x_learner['m0'].fit(train.query(f"{T}==0")[X], train.query(f"{T}==0")[Y])
x_learner['m1'].fit(train.query(f"{T}==1")[X], train.query(f"{T}==1")[Y])

train['pseudo_treated'] = train[Y] - x_learner['m0'].predict(train[X])
train['pseudo_control'] = x_learner['m1'].predict(train[X]) - train[Y]

x_learner['tau0'].fit(train[X], train['pseudo_treated'])
x_learner['tau1'].fit(train[X], train['pseudo_control'])

x_cate_education = {}
x_cate_age = {}

for category, label in education_categories.items():
    subgroup = test[test['eeducate'] == category]
    x_cate_education[label] = (
        (x_learner['tau1'].predict(subgroup[X]) + x_learner['tau0'].predict(subgroup[X])) / 2
    ).mean()

for label, (min_age, max_age) in age_categories.items():
    subgroup = test[(test['tage'] >= min_age) & (test['tage'] <= max_age)]
    x_cate_age[label] = (
        (x_learner['tau1'].predict(subgroup[X]) + x_learner['tau0'].predict(subgroup[X])) / 2
    ).mean()


print("CATE Estimates by Education Category")
print("S-Learner:", s_cate_education)
print("T-Learner:", t_cate_education)
print("X-Learner:", x_cate_education)

print("\nCATE Estimates by Age Category")
print("S-Learner:", s_cate_age)
print("T-Learner:", t_cate_age)
print("X-Learner:", x_cate_age)